In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import glob

# Load all 24 CSV files from LONDON 2024 folder
folder_path = '/content/drive/MyDrive/LONDON 2024/*.csv'
all_files = glob.glob(folder_path)

print(f"Found {len(all_files)} files\n")

# Combine all files
df_list = []
for idx, file in enumerate(all_files, 1):
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)
    print(f"✓ [{idx}/24] {file.split('/')[-1]}: {len(temp_df):,} rows")

# Combine into one dataframe
df = pd.concat(df_list, ignore_index=True)
print(f"\n✓ TOTAL: {len(df):,} rows, {len(df.columns)} columns\n")

# Show preview
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df['Start date'] = pd.to_datetime(df['Start date'], dayfirst=True, format='mixed')
df['End date'] = pd.to_datetime(df['End date'], dayfirst=True, format='mixed')
df['hour'] = df['Start date'].dt.hour

In [ ]:
df['duration_minutes'] = df['Total duration (ms)'] / (1000*60)
print(df['duration_minutes'].mean())
print(df['duration_minutes'].median())

In [ ]:
df['Start station'].value_counts().head(5)


In [ ]:
df['hour'].value_counts().sort_index().plot(kind='bar', figsize=(12,4))


In [ ]:
df['duration_minutes'].plot(kind='hist', bins=40, figsize=(10,4))


In [ ]:
df['weekday'] = df['Start date'].dt.day_name()
df['weekday'].value_counts().plot(kind='bar', figsize=(10,4))


In [ ]:
# Calculating descriptive statistics including standard deviation

mean_duration = df['duration_minutes'].mean()
median_duration = df['duration_minutes'].median()
std_duration = df['duration_minutes'].std()

print("Trip Duration Statistics:")
print(f"Mean: {mean_duration:.2f} minutes")
print(f"Median: {median_duration:.2f} minutes")
print(f"Standard Deviation: {std_duration:.2f} minutes")

In [ ]:
import matplotlib.pyplot as plt

# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(df['duration_minutes'], bins=60, range=(0, 60), edgecolor='black')

# Add mean and median lines
plt.axvline(mean_duration, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_duration:.1f} min')
plt.axvline(median_duration, color='green', linestyle='--', linewidth=2, label=f'Median: {median_duration:.1f} min')

# Labels
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Number of Trips')
plt.title('Distribution of Trip Durations - August 2023')
plt.legend()
plt.grid(True, alpha=0.3)

# Save the figure
plt.savefig('figure1_duration_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved as: figure1_duration_distribution.png")

In [ ]:
# Calculate trips per hour
hourly_counts = df['hour'].value_counts().sort_index()

# Create line plot
plt.figure(figsize=(12, 6))
plt.plot(hourly_counts.index, hourly_counts.values, marker='o', linewidth=2)

# Labels
plt.xlabel('Hour of Day')
plt.ylabel('Number of Trips')
plt.title('Cycle Hire Usage by Hour of Day')
plt.xticks(range(0, 24))
plt.grid(True, alpha=0.3)

# Save
plt.savefig('figure2_hourly_usage.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved as: figure2_hourly_usage.png")

In [ ]:
# Calculate trips per day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_counts = df['weekday'].value_counts().reindex(day_order)

# Create bar chart
plt.figure(figsize=(10, 6))
plt.bar(day_order, daily_counts.values, edgecolor='black')

# Labels
plt.xlabel('Day of Week')
plt.ylabel('Number of Trips')
plt.title('Cycle Hire Usage by Day of Week')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Save
plt.savefig('figure3_weekly_usage.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure saved as: figure3_weekly_usage.png")

In [ ]:
print("TASK 2: DATA QUALITY CHECKS")

# Check missing values
print("\nMissing Values Check:")
print(df.isnull().sum())

# Check duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Check outliers
long_trips = df[df['duration_minutes'] > 120]
print(f"\nTrips over 2 hours: {len(long_trips)} ({len(long_trips)/len(df)*100:.2f}%)")

# Date range - FIXED COLUMN NAME
print(f"\nDate range: {df['Start date'].min()} to {df['End date'].max()}")

# Unique stations - FIXED COLUMN NAME
print(f"\nUnique start stations: {df['Start station'].nunique()}")


In [ ]:
# Create clean dataset for modeling
df_clean = df.drop_duplicates()
df_model = df_clean[df_clean['duration_minutes'] <= 120].copy()

print("Dataset sizes:")
print(f"Original: {len(df):,} trips")
print(f"After removing duplicates: {len(df_clean):,} trips")
print(f"Final (removed >2hr trips): {len(df_model):,} trips")

In [ ]:
print("TASK 3: PREDICTIVE MODEL - FEATURE CREATION")

# Create features - FIXED column name
df_model['hour'] = df_model['Start date'].dt.hour
df_model['day_of_week'] = df_model['Start date'].dt.dayofweek
df_model['is_weekend'] = (df_model['day_of_week'] >= 5).astype(int)
df_model['is_rush_hour'] = df_model['hour'].isin([8, 17, 18]).astype(int)

print(f"\nFeatures created:")
print(f"- Hour: 0 to 23")
print(f"- Weekend trips: {df_model['is_weekend'].sum():,}")
print(f"- Rush hour trips: {df_model['is_rush_hour'].sum():,}")
print(f"\n Dataset ready for modeling: {len(df_model):,} trips")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

print("TASK 3: BUILDING LINEAR REGRESSION MODEL")

# Prepare X and y
X = df_model[['hour', 'is_weekend', 'is_rush_hour']]
y = df_model['duration_minutes']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData split:")
print(f"Training: {len(X_train):,} trips")
print(f"Testing: {len(X_test):,} trips")

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

print("\n Model trained successfully!")


In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Calculate metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("MODEL PERFORMANCE")
print(f"\nR² Score: {r2:.4f}")
print(f"  → Explains {r2*100:.2f}% of variance")
print(f"\nRMSE: {rmse:.2f} minutes")
print(f"MAE: {mae:.2f} minutes")
print(f"  → Average error: ±{mae:.1f} minutes")


In [ ]:
# Get coefficients
intercept = model.intercept_
coef_hour = model.coef_[0]
coef_weekend = model.coef_[1]
coef_rush = model.coef_[2]

print("MODEL FORMULA")
print(f"\nDuration = {intercept:.2f} + ({coef_hour:.3f} × Hour)")
print(f"           + ({coef_weekend:.2f} × Weekend)")
print(f"           + ({coef_rush:.2f} × Rush Hour)")

print(f"\nINTERPRETATION")
print(f"Base duration: {intercept:.2f} minutes")
print(f"Hour effect: {coef_hour:+.3f} min/hour")
print(f"Weekend effect: {coef_weekend:+.2f} minutes")
print(f"Rush hour effect: {coef_rush:+.2f} minutes")


In [ ]:
# Sample data
sample_size = 5000
np.random.seed(42)
indices = np.random.choice(len(y_test), size=min(sample_size, len(y_test)), replace=False)
y_test_sample = y_test.iloc[indices]
y_pred_sample = y_pred[indices]

# Create graph
plt.figure(figsize=(10, 7))
plt.scatter(y_test_sample, y_pred_sample, alpha=0.3, s=10)

max_val = max(y_test_sample.max(), y_pred_sample.max())
plt.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Predictions')

plt.xlabel('Actual Duration (minutes)', fontsize=12)
plt.ylabel('Predicted Duration (minutes)', fontsize=12)
plt.title('Actual vs Predicted Trip Duration', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.savefig('figure4_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: figure4_actual_vs_predicted.png")

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Create graph
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.3, s=10)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)

plt.xlabel('Predicted Duration (minutes)', fontsize=12)
plt.ylabel('Residuals (Actual - Predicted)', fontsize=12)
plt.title('Residual Plot', fontsize=14)
plt.grid(True, alpha=0.3)

plt.savefig('figure5_residuals.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: figure5_residuals.png")

In [ ]:
# Create feature importance graph
features = ['Hour of Day', 'Weekend', 'Rush Hour']
importance = [abs(coef_hour), abs(coef_weekend), abs(coef_rush)]

plt.figure(figsize=(10, 6))
plt.barh(features, importance, edgecolor='black')

plt.xlabel('Absolute Coefficient Value', fontsize=12)
plt.title('Feature Importance', fontsize=14)
plt.grid(True, alpha=0.3, axis='x')

plt.savefig('figure6_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: figure6_feature_importance.png")

In [ ]:
#WEEKDAY VS WEEKEND PATTERNS

print("CREATING WEEKDAY VS WEEKEND COMPARISON")

# Calculate average duration by hour and weekend status
pattern_data = df_model.groupby(['hour', 'is_weekend'])['duration_minutes'].mean().reset_index()
pivot_table = pattern_data.pivot(index='hour', columns='is_weekend', values='duration_minutes')

# Create comparison plot
plt.figure(figsize=(12, 6))
plt.plot(pivot_table.index, pivot_table[0], marker='o', label='Weekday', linewidth=2, markersize=8)
plt.plot(pivot_table.index, pivot_table[1], marker='s', label='Weekend', linewidth=2, markersize=8)

plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Trip Duration (minutes)', fontsize=12)
plt.title('Trip Duration Patterns: Weekday vs Weekend by Hour', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24))

# Highlight peak hours
plt.axvspan(7, 9, alpha=0.1, color='red', label='Morning Rush')
plt.axvspan(17, 19, alpha=0.1, color='orange', label='Evening Rush')

plt.savefig('figure7_weekday_weekend_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: figure7_weekday_weekend_comparison.png")

# Print insights
weekday_avg = df_model[df_model['is_weekend']==0]['duration_minutes'].mean()
weekend_avg = df_model[df_model['is_weekend']==1]['duration_minutes'].mean()
print(f"\nINSIGHTS:")
print(f"Average weekday trip: {weekday_avg:.2f} minutes")
print(f"Average weekend trip: {weekend_avg:.2f} minutes")
print(f"Weekend trips are {weekend_avg - weekday_avg:+.2f} minutes longer")

In [ ]:
# MODEL VALIDATION: CROSS-VALIDATION

from sklearn.model_selection import cross_val_score
import numpy as np

print("PERFORMING 5-FOLD CROSS-VALIDATION")
print("This tests if our model is robust and not just overfitted to our training data\n")

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

print("Cross-Validation Results:")
print(f"Fold 1 R²: {cv_scores[0]:.4f}")
print(f"Fold 2 R²: {cv_scores[1]:.4f}")
print(f"Fold 3 R²: {cv_scores[2]:.4f}")
print(f"Fold 4 R²: {cv_scores[3]:.4f}")
print(f"Fold 5 R²: {cv_scores[4]:.4f}")

print(f"\nAverage R²: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")

# Visualize CV scores
plt.figure(figsize=(10, 6))
plt.bar(range(1, 6), cv_scores, edgecolor='black', alpha=0.7)
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {cv_scores.mean():.4f}')
plt.xlabel('Fold Number', fontsize=12)
plt.ylabel('R² Score', fontsize=12)
plt.title('Cross-Validation R² Scores Across 5 Folds', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.xticks(range(1, 6))

plt.savefig('figure8_cross_validation.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Saved: figure8_cross_validation.png")

print("\nINTERPRETATION:")
print("Consistent scores across folds indicate the model is stable and not overfitted.")
print(f"Our model maintains an R² of ~{cv_scores.mean():.3f} across different data subsets.")

In [ ]:
# BONUS ANALYSIS: PEAK HOUR CHARACTERISTICS


print("ANALYZING PEAK HOUR CHARACTERISTICS")

# Define peak hours
morning_peak = df_model[df_model['hour'].isin([7, 8, 9])]
evening_peak = df_model[df_model['hour'].isin([17, 18, 19])]
off_peak = df_model[~df_model['hour'].isin([7, 8, 9, 17, 18, 19])]

# Calculate statistics
stats_data = {
    'Period': ['Morning Peak (7-9am)', 'Evening Peak (5-7pm)', 'Off-Peak'],
    'Trips': [len(morning_peak), len(evening_peak), len(off_peak)],
    'Avg Duration': [morning_peak['duration_minutes'].mean(),
                     evening_peak['duration_minutes'].mean(),
                     off_peak['duration_minutes'].mean()],
    'Trips per Hour': [len(morning_peak)/3, len(evening_peak)/3, len(off_peak)/18]
}

peak_df = pd.DataFrame(stats_data)
print("\nPeak Period Statistics:")
print(peak_df.to_string(index=False))

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Number of trips
ax1.bar(peak_df['Period'], peak_df['Trips'], edgecolor='black', alpha=0.7)
ax1.set_ylabel('Total Number of Trips', fontsize=11)
ax1.set_title('Trip Volume by Period', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=15)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Average duration
ax2.bar(peak_df['Period'], peak_df['Avg Duration'], color='orange', edgecolor='black', alpha=0.7)
ax2.set_ylabel('Average Duration (minutes)', fontsize=11)
ax2.set_title('Average Trip Duration by Period', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=15)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('figure9_peak_hour_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Saved: figure9_peak_hour_analysis.png")

print("\nKEY TAKEAWAYS FOR EXETER:")
print(f"1. Morning peak sees {len(morning_peak):,} trips (avg {morning_peak['duration_minutes'].mean():.1f} min)")
print(f"2. Evening peak sees {len(evening_peak):,} trips (avg {evening_peak['duration_minutes'].mean():.1f} min)")
print(f"3. Peak hours account for {(len(morning_peak)+len(evening_peak))/len(df_model)*100:.1f}% of all trips")


In [ ]:
print("FINAL SUMMARY - COPY THESE INTO YOUR REPORT")
print("\nTASK 1 STATISTICS:")
print(f"Mean: {mean_duration:.2f} minutes")
print(f"Median: {median_duration:.2f} minutes")
print(f"Std Dev: {std_duration:.2f} minutes")
print(f"Total trips: {len(df):,}")

print("\nTASK 2 DATA CLEANING:")
print(f"Original: {len(df):,} trips")
print(f"Final clean: {len(df_model):,} trips")
print(f"Percentage retained: {len(df_model)/len(df)*100:.1f}%")

print("\nTASK 3 MODEL RESULTS:")
print(f"R²: {r2:.4f} ({r2*100:.2f}% variance explained)")
print(f"MAE: {mae:.2f} minutes")
print(f"RMSE: {rmse:.2f} minutes")
print(f"\nModel coefficients:")
print(f"Intercept: {intercept:.2f}")
print(f"Hour: {coef_hour:.3f}")
print(f"Weekend: {coef_weekend:.2f}")
print(f"Rush hour: {coef_rush:.2f}")


In [ ]:
# Create clean dataset for Power BI
df_powerbi = df[['Start date', 'hour', 'weekday', 'duration_minutes', 'Start station']].copy()

# Save to CSV
df_powerbi.to_csv('london_cycles_powerbi.csv', index=False)

# Download the file
from google.colab import files
files.download('london_cycles_powerbi.csv')